In [1]:
import duckdb
import pandas as pd
from IPython.display import display

pd.set_option('display.max_columns', None)

In [2]:

# Path to DuckDB database
DB_PATH = '../../database/clinisys_all.duckdb'

use_schema = 'silver'

# List of relevant silver tables
silver_tables = [
    'view_micromanipulacao',
    'view_congelamentos_ovulos',
    'view_descongelamentos_ovulos',
    'view_congelamentos_embrioes',
    'view_embrioes_congelados',
    'view_descongelamentos_embrioes',
    'view_tratamentos',
    # 'view_micromanipulacao_oocitos',
]

# Table that needs special handling
micromanip_oocitos_table = 'view_micromanipulacao_oocitos'


# Prompt for prontuario
prontuario = '160751' #'160751' #'171805'

# Connect to DuckDB
db = duckdb.connect(DB_PATH)

# Special handling for micromanipulacao_oocitos
print(f'\n--- Records from {use_schema}.{micromanip_oocitos_table} for prontuario={prontuario} (via view_micromanipulacao) ---')
try:
    # Get all codigo_ficha for this prontuario
    codigos = db.execute(f"""
        SELECT codigo_ficha FROM {use_schema}.view_micromanipulacao WHERE prontuario = ?
    """, [prontuario]).fetchall()
    codigo_list = [row[0] for row in codigos]
    if codigo_list:
        # Query oocitos table for all id_micromanipulacao in codigo_list
        placeholders = ','.join(['?']*len(codigo_list))
        query = f"""
            SELECT * FROM {use_schema}.{micromanip_oocitos_table}
            WHERE id_micromanipulacao IN ({placeholders})
            ORDER BY id_micromanipulacao, id
        """
        df = db.execute(query, codigo_list).df()
        if not df.empty:
            display(df)
        else:
            print('No records found.')
    else:
        print('No matching codigo_ficha found in view_micromanipulacao.')
except Exception as e:
    print(f'Error querying {use_schema}.{micromanip_oocitos_table}:', e)


for table in silver_tables:
    print(f'\n--- Records from {use_schema}.{table} for prontuario={prontuario} ---')
    # Some tables may not have prontuario, so try/except
    try:
        df = db.execute(f"""
            SELECT * FROM {use_schema}.{table}
            WHERE prontuario = ?
            order by 1
        """, [prontuario]).df()
        if not df.empty:
            print(f'Shape: {df.shape}')
            display(df)
        else:
            print('No records found.')
    except Exception as e:
        print(f'Error querying {use_schema}.{table}:', e)

db.close()


--- Records from silver.view_micromanipulacao_oocitos for prontuario=160751 (via view_micromanipulacao) ---


,id,id_micromanipulacao,diaseguinte,Maturidade,RC,ComentariosAntes,Embriologista,PI,TCD,AH,PGD,ResultadoPGD,IdentificacaoPGD,Fertilizacao,CorpusculoPolar,ComentariosDepois,GD1,OocitoDoado,ICSI,OrigemOocito,InseminacaoOocito,NClivou_D2,NCelulas_D2,Frag_D2,Blastomero_D2,NClivou_D3,NCelulas_D3,Frag_D3,Blastomero_D3,GD2,NClivou_D4,NCelulas_D4,Compactando_D4,MassaInterna_D4,Trofoblasto_D4,NClivou_D5,NCelulas_D5,Compactando_D5,MassaInterna_D5,Trofoblasto_D5,NClivou_D6,NCelulas_D6,Compactando_D6,MassaInterna_D6,Trofoblasto_D6,NClivou_D7,NCelulas_D7,Compactando_D7,MassaInterna_D7,Trofoblasto_D7,score_maia,relatorio_ia,hash,extraction_timestamp,flag_embryoscope,embryo_number
0,105080,12102,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,C1,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,edc82079d77a7df6b2366c435da905d4,2025-07-21 21:40:02,0,<NA>
1,105081,12102,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,C1,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,7f4fe67dc5667b3b95859bc0778d97ae,2025-07-21 21:40:02,0,<NA>
2,105082,12102,Não,MII,None,None,None,None,Criopreservado,None,None,None,None,None,C1,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,759502c2f2b7d06b6271950ba22d7cb0,2025-07-21 21:40:02,0,<NA>
3,105083,12102,Não,GV,None,None,None,None,Descartado,None,None,None,None,None,C1,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,eb7fcedcb948086b11b0fbd839315d8d,2025-07-21 21:40:02,0,<NA>
4,105084,12102,Não,GV,None,None,None,None,Descartado,None,None,None,None,None,C1,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,415a63fbb69b83a7906b9f60133bef59,2025-07-21 21:40:02,0,<NA>
5,105085,12102,Não,GV,None,None,None,None,Descartado,None,None,None,None,None,C0,None,None,Não,None,Fresco,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0bb32c8feb217cb3080bcb41d01e099c,2025-07-21 21:40:02,0,<NA>
6,265872,26549,Não,MII,None,None,3548,None,Descartado,None,None,None,None,1CP,None,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,97371e4d615aca722ab027fa28adc566,2025-07-28 19:05:31,1,1
7,265873,26549,Não,MII,None,None,3548,None,Descartado,None,None,None,None,2PN,C1,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,Sim,6,2,Simétricos,None,None,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,a322eaa21774429503e362e662e1026d,2025-07-28 19:05:31,1,2
8,265874,26549,Não,MII,None,None,3548,None,Descartado,None,None,None,None,2PN,C1,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,3c746a6ebccc206306a1a5f31d11cf70,2025-07-28 19:05:31,1,3
9,265875,26549,Não,MII,None,None,3548,3492,Criopreservado,Sim,Sim,None,PGT-A,2PN,C0,Fresco,None,Não,None,Fresco,ICSI,None,None,None,None,Sim,7,1,Simétricos,None,None,None,None,None,None,Sim,None,Blastocisto Grau 1,None,None,Sim,None,Blastocisto Grau 3,C,C,None,None,None,None,None,None,None,36cedad6b4dc5eaced6e732257207723,2025-07-29 19:05:28,1,4



--- Records from silver.view_micromanipulacao for prontuario=160751 ---
Shape: (2, 211)


,codigo_ficha,numero_caso,prontuario,IdadeEsposa_DG,IdadeMarido_DG,Data_DL,codigo_congelamento_semen,horario_inicial_fert,horario_final_fert,Aspiracao_DL,medico_id,responsavel_puncao_id,abstinencia_recomendada,recepcao_ovulos,CicloDoadora,IdadeDoadora,checagem_d4,Denud_DL,checagem_fertilizacao,checagem_d2,checagem_d3,checagem_d5,checagem_d6,checagem_d7,foliculos,oocitos,sangue_cateter_interno,dificuldade_transf,retorno_embrioes,vezes_retorno,sangue_cateter_externo,ICSIDescongelados,CicloDescongelamento,pgd_oocito,pgd_d3,pgd_d5,data_procedimento,tecnica_injetados,motilidade,morfologia_injetados,gradiente_descontinuo,mini_gradiente,data_procedimento_2,tecnica_injetados_2,motilidade_2,morfologia_injetados_2,unidade_id,data_microtese,data_microtese_2,validade7,responsavel_denud_id,profissional_fertilizacao_id,profissional_d2_id,profissional_d3_id,profissional_d5_id,profissional_d6_id,profissional_d7_id,validade11,materiais_insumos,local_coleta,fonte_outros,gradiente_descontinuo_2,viscosidade,volume_amostra_utilizada,morfologia_pre,morfologia_pos,zymot,mini_gradiente_2,local_coleta_2,fonte_outros_2,observacoes_resumo_ciclos,viscosidade_2,volume_amostra_utilizada_2,morfologia_pre_2,morfologia_pos_2,zymot_2,validade20,gradiente,lote21,validade21,agulha_icsi,lote_agulha_icsi,validade_agulha_icsi,agulha_aspir,lote_agulha_aspir,validade_agulha_aspir,incubadora,abstinencia_dias,meios,responsavel_oocitos_id,fonte,inicio_ps,termino_ps,volume_ps1,CongelamentoOvulos,concentracao_progressivos_pre,c_ps1,d_ps1,conc_ps1,diametro_minimo_pse,diametro_medio_pse,diametro_maximo_pse,tubulos_avaliados_pse,peso_total_pse,peso_microtese_pse,peso_biopsia_pse,diametro_minimo_psd,diametro_medio_psd,diametro_maximo_psd,tubulos_avaliados_psd,peso_total_psd,peso_microtese_psd,peso_biopsia_psd,responsavel_transfer_id,MorfologiaAlterada,EstadoSptz,CongelamentoSemenHom,VialsDescongeladasHom,CongelamentoSemenHet,VialsDescongeladasHet,check_gradiente_ps,concentrado,check_swimup,check_lavagem,volume_ps2,MotivoCongelamentoOvulos,concentracao_progressivos_pos,c_ps2,d_ps2,conc_ps2,comentarios_ps,responsavel_ps,fonte2,inicio_ps2,termino_ps2,volume_ps3,concentracao_progressivos_pre_2,c_ps3,d_ps3,conc_ps3,diametro_minimo_pse2,diametro_medio_pse2,diametro_maximo_pse2,tubulos_avaliados_pse2,peso_total_pse2,peso_microtese_pse2,peso_biopsia_pse2,diametro_minimo_psd2,diametro_medio_psd2,diametro_maximo_psd2,tubulos_avaliados_psd2,peso_total_psd2,peso_microtese_psd2,peso_biopsia_psd2,timelapse,EstadoSptz2,codigo_congelamento_semen_2,CongelamentoSemenHom2,VialsDescongeladasHom2,CongelamentoSemenHet2,VialsDescongeladasHet2,check_gradiente_ps2,concentrado_2,check_swimup2,check_lavagem2,volume_ps4,concentracao_progressivos_pos_2,c_ps4,d_ps4,conc_ps4,comentarios_ps2,responsavel_ps2,transferidos,cateter,lote_cateter,validade_cateter,motivo_nao_transferencia,Transfer_D5,medico_responsavel_id,azoospermia,cod1_crio,cod2_crio,tanque,cane,tecnica,Observacoes,responsavel_labfiv,responsavel_labfiv_data,tipo_paciente,responsavel_biopsia,alteracoes_oocitarias_normal,alteracoes_oocitarias_granulacao,alteracoes_oocitarias_pelucida,alteracoes_oocitarias_oval,alteracoes_oocitarias_inclusoes,alteracoes_oocitarias_pervitelineo_aumentado,alteracoes_oocitarias_irregular,alteracoes_oocitarias_necrotica,alteracoes_oocitarias_pervitelineo_granular,alteracoes_oocitarias_ser,alteracoes_oocitarias_corpusculo_frag,alteracoes_oocitarias_vacuolos,alteracoes_oocitarias_corpusculo_aum,alteracoes_oocitarias_membrana,alteracoes_oocitarias_corpusculo_peq,alteracoes_oocitarias_citoplasma,alteracoes_oocitarias_corpusculo_deg,controle_anual,maia,hash,extraction_timestamp
0,12102,IB1414/22,160751,39,NaN,2022-09-25,None,None,None,10:20,10,3531,None,Não,None,None,None,12:00,None,None,None,None,None,None,7,6,None,None,None,None,None,None,None,None,None,None,NaT,None,None,None,None,None,None,None,None,None,0000000001,<NA>,<NA>,<NA>,4005,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,None,None,


--- Records from silver.view_congelamentos_ovulos for prontuario=160751 ---
Shape: (2, 30)


,id,CodCongelamento,Unidade,prontuario,paciente,Data,Hora,Ciclo,condicoes_amostra,empresa_transporte,clinica_origem,responsavel_recebimento,responsavel_recebimento_data,responsavel_armazenamento,responsavel_armazenamento_data,NOvulos,NPailletes,Identificacao,Tambor,Cane,Cane2,Tecnica,Motivo,Observacoes,BiologoResponsavel,BiologoFIV,BiologoFIV2,status_financeiro,hash,extraction_timestamp
0,8363,EGG3902,1,160751,esposa,2022-09-25,1900-01-01 12:25:00,IB1414/22,None,None,None,<NA>,NaT,<NA>,NaT,3,1,EGG3902,AD,5,None,Vitrificação,Preservação da Fertilidade,None,<NA>,Thiago Placido,Bruna Lázaro Lourenço,None,fb1f4501c3b7920107f74853f778a03f,2025-07-21 21:38:49
1,10329,EGG2325,1,160751,esposa,2018-07-02,1900-01-01 10:00:00,-,None,None,None,<NA>,NaT,<NA>,NaT,13,4,EGG2325,EPSILON,6,None,Vitrificação,Preservação da Fertilidade,None,<NA>,Mariana Nicolielo,Mariana Nicolielo,None,f1856c989574c6aefd294925f3831a53,2025-07-21 21:38:49



--- Records from silver.view_descongelamentos_ovulos for prontuario=160751 ---
No records found.

--- Records from silver.view_congelamentos_embrioes for prontuario=160751 ---
No records found.

--- Records from silver.view_embrioes_congelados for prontuario=160751 ---
No records found.

--- Records from silver.view_descongelamentos_embrioes for prontuario=160751 ---
No records found.

--- Records from silver.view_tratamentos for prontuario=160751 ---
No records found.
